# Notebook pour préparer l'union des niveaux pression et la creation des axes/zoom appropriés

In [1]:
# Select your laboratory among: 'cnrm', 'cerfacs', 'ipsl'
lab='cerfacs'


## Lab settings

In [2]:
if lab=='cnrm' or lab=='cerfacs':
    # This dictionnary should be the same as the one used for creating file_defs.
    # Here , we quoted only those entries useful for creating ping files
    settings={
        #'mips' : {'AerChemMIP','C4MIP','CFMIP','DAMIP', 'FAFMIP' , 'GeoMIP','GMMIP','ISMIP6',\
        #                  'LS3MIP','LUMIP','OMIP','PMIP','RFMIP','ScenarioMIP','CORDEX','SIMIP'},
        # If you want to get comprehensive ping files; use :
        'mips' : {"CMIP6", "AerChemMIP", "C4MIP", "CFMIP", "DAMIP", "DCPP", "FAFMIP", "GeoMIP", "GMMIP", 
                  "HighResMIP", "ISMIP6", "LS3MIP", "LUMIP", "OMIP", "PDRMIP", "PMIP", "RFMIP", "ScenarioMIP", 
                  "SolarMIP", "VolMIP", "CORDEX", "DynVar", "SIMIP", "VIACSAB", "SPECS", "CCMI", "CMIP5", 
                  "CMIP", "DECK"},
        'max_priority' : 3,
        'tierMax'      : 3,
       # Each XIOS  context does adress a number of realms
        'realms_per_context' : { 'nemo': ['seaIce', 'ocean', 'ocean seaIce', 'ocnBgchem', 'seaIce ocean'] ,
                              'arpsfx' : ['atmos', 'atmos atmosChem', 'aerosol', 'atmos land', 'land',
                                         'landIce land',  'aerosol land','land landIce',  'landIce', ],
                              }, 
        "ping_variables_prefix" : "CMIP6_",
        # We account for a file listing the variables which the lab does not want to produce 
        # Format : MIP varname as first column, comment lines begin with '#'
        #"excluded_vars_file":"/cnrm/est/USERS/senesi/public/CMIP6/data_request/cnrm/excluded_vars.txt",
        "excluded_vars_file" : None,
        "excluded_vars" : [],
        # We account for a list of variables which the lab wants to produce in some cases
        "listof_home_vars":None,
        #"listof_home_vars": None,
        "path_extra_tables":None,
        }
    

In [3]:
if lab=='cerfacs':  
    #settings["mips"]={'HighResMIP','DCPP'}
    settings["listof_home_vars"]="./inputs/my_listof_home_vars.txt"
    settings["path_extra_tables"]="./inputs/extra_Tables"
    

In [4]:
if lab=='ipsl':
    # This dictionnary should be the same as the one used for creating file_defs.
    # Here , we quoted only those entries useful for creating ping files
    settings={
        #'mips' : {'AerChemMIP','C4MIP','CFMIP','DAMIP', 'FAFMIP' , 'GeoMIP','GMMIP','ISMIP6',\
        #                  'LS3MIP','LUMIP','OMIP','PMIP','RFMIP','ScenarioMIP','CORDEX','SIMIP'},
        # If you want to get comprehensive ping files; use :
        'mips' : {"CMIP6", "AerChemMIP", "C4MIP", "CFMIP", "DAMIP", "DCPP", "FAFMIP", "GeoMIP", "GMMIP", 
                  "HighResMIP", "ISMIP6", "LS3MIP", "LUMIP", "OMIP", "PDRMIP", "PMIP", "RFMIP", "ScenarioMIP", 
                  "SolarMIP", "VolMIP", "CORDEX", "DynVar", "SIMIP", "VIACSAB", "SPECS", "CCMI", "CMIP5", 
                  "CMIP", "DECK"},
        'max_priority' : 3,
        'tierMax'      : 3,
       # Each XIOS  context does adress a number of realms
        'realms_per_context' : { 
            'nemo': ['seaIce', 'ocean', 'ocean seaIce', 'ocnBgchem', 'seaIce ocean'],
            'lmdz' : ['atmos', 'atmos land'] , 
            'orchidee': ['land', 'landIce land',  'land landIce', 'landIce'] ,
                              }, 
        "ping_variables_prefix" : "CMIP6_",
        # We account for a file listing the variables which the lab does not want to produce 
        # Format : MIP varname as first column, comment lines begin with '#'
        "excluded_vars_file" : None,
        "excluded_vars" : None,
        # We account for a list of variables which the lab wants to produce in some cases
        "listof_home_vars":None,
        "path_extra_tables":None,
        } 
    

In [5]:
from dr2xml import gather_AllSimpleVars, isVertDim #, create_axis_def
from vars import simple_Dim

01.00.04


## Select all variables to consider, based on lab settings

In [6]:
# Attention: appel a select_CMORvars_for_lab sous-jacent sans filtre par experience
svars=gather_AllSimpleVars(settings,printout=False)


DR Error: issue with dimids for  zhalfo in Table  Omon  => no sdims derived.
DR Error: issue with mipvar for sic  => no label_without_area, long_name, stdname, description and units derived.
DR Error: issue with dimids for  zhalfo in Table  Omon  => no sdims derived.
DR Error: issue with mipvar for siltfrac  => no label_without_area, long_name, stdname, description and units derived.
DR Error: issue with mipvar for sandfrac  => no label_without_area, long_name, stdname, description and units derived.
DR Error: issue with mipvar for clayfrac  => no label_without_area, long_name, stdname, description and units derived.
DR Error: issue with mipvar for hfdsnt  => no label_without_area, long_name, stdname, description and units derived.

Number of simplified vars is : 3560
Info: dimid corresponding to  height50m for variable wind50mmin in Table PRIMAVERA_Prim1hr  not found in DR => read it in extra coordinates Table:  height 
Info: dimid corresponding to  height50m for variable va50m in Tab

# Nouvelles fonctions  (ou évolution d'anciennes) pour union/zoom

In [7]:
# Fonction create_xios_aux_elmts_defs non utilisee dans ce notebook, juste preparee pour implantation dans dr2xml
# mpmoine_union_modif: renommage de la fonction 'create_xios_field_ref' en 'create_xios_aux_elmts_defs'
def create_xios_aux_elmts_defs(sv,alias,table,lset,sset,end_field_defs,
    field_defs,axis_defs,domain_defs,dummies,context,remap_domain,pingvars) :
    """
    Create a field_ref for a simplified variable object sv (with
    lab prefix for the variable name) and store it in end_field_defs
    under a key=shape
    
    Add field definitions for intermediate variables in dic field_defs
    Add axis  definitions for interpolations in dic axis_defs
    Use pingvars as the list of variables actually defined in ping file

    """
    # By convention, field references are built as prefix_<MIP_variable_name>
    # Such references must be fulfilled using a dedicated filed_def
    # section implementing the match between legacy model field names
    # and such names, called 'ping section'
    #
    # Identify which 'ping' variable ultimatley matches the requested
    # CMOR variable, based on shapes. This may involve building
    # intermediate variables, in order to  apply corresponding operations

    # The preferred order of operation is : vertical interp (which
    # is time-dependant), time-averaging, horizontal operations (using
    # expr=@this)
    
    # nextvar is the field name provided as output of the last
    # operation currently defined
    # mpmoine_union_modif:create_xios_field_ref: on supprime l'usage de netxvar
    #
    #--------------------------------------------------------------------
    # Build XIOS axis elements (stored in axis_defs)
    # Proceed with vertical interpolation if needed
    #---
    # Build XIOS auxilliary field elements (stored in field_defs)
    #--------------------------------------------------------------------
    ssh=sv.spatial_shp
    prefix=lset["ping_variables_prefix"]
    # mpmoine_union_modif:create_xios_field_ref: recup de lwps
    lwps=sv.label_without_psuffix
    # TBD Should handle singletons here
    # TBD Should ensure that various additionnal dims are duly documented by model or pingfile (e.g. tau)
    if ssh[0:4] in ['XY-H','XY-P'] or ssh[0:3] == 'Y-P' :
        # TBD : for now, do not interpolate vertically
        # mpmoine_temporaire:je reactive l'ecriture des axis_def: return
        # mpmoine_last_modif:create_xios_field_ref: on recupere maintenant 'dimids' depuis svar
        # mpmoine_future_modif:create_xios_field_ref: on utilise maintenant sv.sdims pour analyser les dimension
        # mpmoine_question: je ne comprend pas l'usage de nextvar... A priori on ne peut pas avoir plus d'une dimension verticale ?
        for sd in sv.sdims.values():
            if isVertDim(sd):
                # mpmoine_union_modif:create_xios_field_ref: on supprime l'usage de netxvar
                # mpmoine_union_modif:create_xios_field_ref: passage par 2 niveaux de field id auxiliaires rebond (alias et alias2)
                if sd.is_zoom_of:
                    alias+="_"+sd.label
                    alias2=prefix+"_"+lwps+"_union"
                    cible=prefix+"_"+lwps
                    axis_key=sd.zoom_label 
                else:
                    alias+="_"+sd.label
                    alias2=False
                    cible=prefix+"_"+lwps
                    axis_key=sd.label 
                if not cible in pingvars:
                    print "Warning: field id",cible,"expected in pingfile but not found."
                if not alias in pingvars:
                    # Construct an axis for interpolating to this dimension
                    # mpmoine_future_modif:create_xios_field_ref: suppression de l'argument 'field_defs' de create_axis_def qui n'est pas utilise
                    # only zoom or normal axis attached to svar, axis for unions of plevs are managed elsewhere
                    axis_defs[axis_key]=create_axis_def(sd,lwps,lset["ping_variables_prefix"])
                    # Construct a field def for the interpolated variable
                    if alias2:
                        field_defs[alias]='<field id="%-25s field_ref="%-25s axis_ref="%-10s/>'\
                        %(alias+'"',alias2+'"',sd.zoom_label+'"')
                        field_defs[alias2]='<field id="%-25s field_ref="%-25s axis_ref="%-10s/>'\
                        %(alias2+'"',cible+'"',sd.is_zoom_of+'"')
                    else:
                        field_defs[alias]='<field id="%-25s field_ref="%-25s axis_ref="%-10s/>'\
                        %(alias+'"',cible+'"',sd.label+'"')                    
                #TBD what to do for singleton dimension ?
    #
    #--------------------------------------------------------------------
    # Build XIOS domain elements (stored in domain_defs)
    #--------------------------------------------------------------------
     # Analyze 'outermost' time cell_method and translate to 'operation'
    operation,detect_missing = analyze_cell_time_method(sv.cell_methods,sv.label,table)
    # Horizontal operations. Can include horiz re-gridding specification
    # Compute domain name, define it if needed
    domain_ref=None
    if ssh[0:2] == 'Y-' : #zonal mean and atm zonal mean on pressure levels
        # TBD should remap before zona mean
        domain_ref="zonal_mean"
        domain_defs[domain_ref]='<domain id="%s"/>'%domain_ref
    elif ssh[0:2] == 'S-' : #COSP sites; cas S-na, S-A, S-AH
        domain_ref="COSP_sites"
        domain_defs[domain_ref]='<domain id="%s"/>'%domain_ref
    elif ssh[0:2] == 'L-' :
        domain_ref="COSP_curtain"
        domain_defs[domain_ref]='<domain id="%s"/>'%domain_ref
    elif ssh == 'TR-na' or ssh == 'TRS-na' : #transects,   oce or SI
        pass
    elif ssh[0:3] == 'XY-'  : # includes 'XY-AH' : model half-levels
        if remap_domain : domain_ref=remap_domain
    elif ssh[0:3] == 'YB-'  : #basin zonal mean or section
        pass
    elif ssh      == 'na-na'  : # global means or constants
        pass 
    else :
        raise(dr2xml_error("Issue with un-managed spatial shape %s"%ssh))
    #
    #--------------------------------------------------------------------
    # Build XIOS field elements (stored in end_field_defs)
    # including their CMOR attributes
    #--------------------------------------------------------------------
    rep='  <field field_ref="%s" name="%s" ts_enabled="true" '% \
        (alias,sv.label)
    rep+=' operation="%s" detect_missing_value="%s" default_value="1.e+20"'% \
        ( operation,detect_missing)
    rep+=' cell_methods="%s" cell_methods_mode="overwrite"'% sv.cell_methods
    rep+='>\n'
    #
    comment=None
    # Process experiment-specific comment for the variable
    if sv.label in sset['comments'] :
        comment=sset['comments'][sv.label] 
    else: # Process lab-specific comment for the variable
        if sv.label in lset['comments'] : 
            comment=sset['comments'][sv.label] 
    if comment : rep+=wrv('comment',comment) #TBI 
    #
    rep+=wrv("standard_name",sv.stdname)
    #
    desc=sv.description
    if desc : desc=desc.replace(">","").replace("<","")
    rep+=wrv("description",desc)
    #
    rep+=wrv("long_name",sv.long_name)
    if sv.positive != "None" and sv.positive != "" :
        rep+=wrv("positive",sv.positive) 
    rep+=wrv('history','none')
    rep+=wrv('units',sv.stdunits)
    # mpmoine_last_modif: ajout de missing_value pour satisfaire le standard attendu par CMOR
    # mpmoine_last_modif: missing_valueS pour l'instant pour que ça passe le CMORchecker (issue)
    rep+=wrv('missing_values',sv.missing,num_type="double")
    rep+=wrv('cell_measures',sv.cell_measures)
    rep+='     </field>\n'
    #
    shape=domain_ref
    #shape=sv.spatial_shp
    if shape not in end_field_defs : end_field_defs[shape]=[]
    end_field_defs[shape].append(rep)

In [8]:
# mpmoine_future_modif:create_axis_def: suppression de l'argument 'field_defs' qui n'est pas utilise
def create_axis_def(sdim,prefix):
    """ 
    From a  simplified Dim object, returns an Xios axis definition 
    """
    # mpmoine_future_modif:create_axis_def: plusieurs modifs car on passe maintenant sdim en argument et non dim_name_or_obj
    if sdim is None:
        print "Warning: cannot create an axis_def from "+sdim
        return None

    # mpmoine_union_modif:create_axis_def: nbre de valeurs de l'axe determine aussi si on est en dim singleton
    if sdim.requested:
        # mpmoine_future_modif: je vire le separateur " ", pour regler le pb des " " successifs
        glo_list=sdim.requested.strip(" ").split()
    else:
        glo_list=sdim.value.strip(" ").split()
    glo_list_num=[float(v) for v in glo_list]
    n_glo=len(glo_list)

    # mpmoine_union_modif:create_axis_def: traitement du cas non zoom (classique, comme avant)
    if not sdim.is_zoom_of:
        # Axis is not a zoom of another, write axis_def normally (with value, interpolate_axis,etc.)
        rep='<axis id="%s" '%sdim.label
        if not sdim.positive in [ None, "" ] :
            rep+='positive="%s" '%sdim.positive
        if n_glo>1 :
            # Case of a non-degenerated dimension (not a singleton)
            rep+='n_glo="%g" '%n_glo
            # mpmoine_future_modif: je supprime le -1 pour n_glo car regle avec rstrip/split()
            rep+='value="(0,%g) [%s]"'%(n_glo,sdim.requested)
        else:
            if n_glo!=1:  
                print "Warning: axis is sigleton but has",n_glo,"values"
                return None
            # Singleton case
            rep+='n_glo=%g '%n_glo
            rep+='value=(0,0)[%s]"'%sdim.value
        rep+=' name="%s"'%sdim.out_name
        rep+=' standard_name="%s"'%sdim.stdname
        rep+=' long_name="%s"'%sdim.long_name
        rep+=' unit="%s"'%sdim.units
        rep+='>'
        if sdim.stdname=="air_pressure" : coordname=prefix+"pfull"
        if sdim.stdname=="altitude"     : coordname=prefix+"zg"
        rep+='\n\t<interpolate_axis type="polynomial" order="1"'
        rep+=' coordinate="%s"/>\n</axis>'%coordname
        return rep
    # mpmoine_union_modif:create_axis_def: traitement du cas zoom
    else:
        # Axis is subset of another, write it as a zoom_axis
        rep='<axis id="%s"'%sdim.zoom_label
        rep+=' axis_ref="%s">\n'%sdim.is_zoom_of
        rep+='\t<zoom_axis begin "%g" n="%g"/>\n'%(glo_list_num[-1],n_glo)
        rep+='</axis>'
        return rep

In [80]:
# mpmoine_future_modif: liste des suffixes de noms de variables reperant un ou plusieurs niveaux pression
#multi_plev_suffixes=set(["10","19","23","27","39","3","3h","4","7c","7h","8","12"])
multi_plev_suffixes=set()
single_plev_suffixes=set(["1000","200","220","500","560","700","840","850","100"])

In [84]:
def create_xios_axis_for_plevs_unions(svars,plev_sfxs,printout=False): 
    """
    Objective of this function is to optimize Xios vertical interpolation requested in pressure levels. 
    Process in 2 steps:
    * First, search pressure levels unions for each simple variable label without psuffix and build a dictionnary :
        dict_plevs is a 3-levels intelaced dictionnary containing for each var (key=svar-label_without_psuffix), 
        the list of svar (key=svar-label,value=svar-object) per pressure levels set (key=sdim-label):
        { "varX":
              { "plevA": {"svar1":svar1,"svar2":svar2,"svar3":svar3},
                "plevB": {"svar4":svar4,"svar5":svar5},
                "plevC": {"svar6":svar6} },
           "varY":
             { "plevA": {"svar7":svar7},
               "plevD": {"svar8":svar8,"svar9":svar9} }
        }
    * Second, create and write Xios union axis (axis id: union_plevs_<label_without_psuffix>)
    """
    #
    #First, search plev unions for each label_without_psuffix and build dict_plevs
    dict_plevs={}
    for sv in svars:
        if not sv.modeling_realm: print "Warning: no modeling_realm associated to:", \
                                            sv.label, sv.mipTable, sv.mip_era
        for sd in sv.sdims.values():
            if sd.label.startswith("p") and any(sd.label.endswith(s) for s in plev_sfxs): # couvre les dimensions verticales de type 'plev7h' ou 'p850'
                lwps=sv.label_without_psuffix
                if lwps:
                    sv.sdims[sd.label].is_zoom_of="union_plevs_"+lwps
                    if not dict_plevs.has_key(lwps):
                        dict_plevs[lwps]={sd.label:{sv.label:sv}}
                    else:
                        if not dict_plevs[lwps].has_key(sd.label):
                            dict_plevs[lwps].update({sd.label:{sv.label:sv}})
                        else:    
                            if sv.label not in dict_plevs[lwps][sd.label].keys(): 
                                dict_plevs[lwps][sd.label].update({sv.label:sv})
                            else:
                                #-print sv.label,"in table",sv.mipTable,"already listed for",sd.label
                                pass
                    # svar will be expected on a zoom axis of the union. Corresponding vertical dim must
                    # have a zoom_label named plevXX_<lwps>
                    sv.sdims[sd.label].zoom_label='zoom'+'_'+sd.label+"_"+lwps 
                    #-print "ZOOM_LABEL>", sv.sdims[sd.label].zoom_label
                else:
                    print "Warning: dim is pressure but label_without_psuffix=", lwps, \
                            "for",sv.label, sv.mipTable, sv.mip_era
    #-for k,v in dict_plevs.items(): print k,v
    #
    # Second, create xios axis for union of plevs
    union_axis_defs={}
    for lwps in dict_plevs.keys():
        sdim_union=simple_Dim()
        plevs_union_xios=""
        plevs_union=set()
        for plev in dict_plevs[lwps].keys():  
            plev_values=[]
            for svar in dict_plevs[lwps][plev].values(): 
                if not plev_values:
                    # svar is the first one with this plev => get its level values
                    # mpmoine_note: on reecrase les attributs de sdim_union à chaque nouveau plev. Pas utile mais
                    # mpmoine_note: c'est la facon la plus simple de faire
                    sdsv=svar.sdims[plev]
                    if sdsv.stdname:   sdim_union.stdname=sdsv.stdname
                    if sdsv.long_name: sdim_union.long_name=sdsv.long_name
                    if sdsv.positive:  sdim_union.positive=sdsv.positive
                    if sdsv.out_name:  sdim_union.out_name=sdsv.out_name
                    if sdsv.units:     sdim_union.units=sdsv.units
                    # case of multi pressure levels
                    plev_values=set(sdsv.requested.split())
                    if not plev_values:
                        # case of single pressure level
                        plev_values=set(sdsv.value.split())
                    plevs_union=plevs_union.union(plev_values)
                    if printout: print "    -- on",plev,":",plev_values 
                if printout: print "       *",svar.label,"(",svar.mipTable,")"
        list_plevs_union=list(plevs_union)
        list_plevs_union.sort(reverse=True)
        for lev in list_plevs_union: plevs_union_xios+=" "+lev
        if printout: print ">>> XIOS plevs union:", plevs_union_xios
        sdim_union.label="union_levs_"+lwps
        sdim_union.requested=plevs_union_xios
        axis_def=create_axis_def(sdim_union,settings["ping_variables_prefix"])
        union_axis_defs.update({sdim_union.label:axis_def})
    return union_axis_defs

# Construction des axes d'union de Plevs

In [85]:
union_axis_defs=create_xios_axis_for_plevs_unions(svars,multi_plev_suffixes.union(single_plev_suffixes))

In [86]:
#for axd in union_axis_defs.values(): print axd

# Construction des axes de zoom

In [87]:
axis_defs={}
for sv in svars:
    lwps=sv.label_without_psuffix
    for sd in sv.sdims.values():
        if isVertDim(sd):
            if sd.is_zoom_of: k=sd.zoom_label 
            else: k=sd.label
            axis_defs[k]=create_axis_def(sd,settings["ping_variables_prefix"])
#for k,v in axis_defs.items(): print k,'\n',v

# Juste quelques tests de devel...

## Pour supprimer les suffixes de pression

In [13]:
# Test
for sv in svars:
    for sd in sv.sdims.values():
        if sd.label.startswith("p"):
            #print "--- ",sv.label
            #print "    * ",sd.label,"axis is zoom of",sd.is_zoom_of
            pass

In [37]:
# Test
plev_suffixes=["1000","200","220","500","560","700","840","850","10","19","23","27","39","3","3h","4","7c","7h","8"]
for s in plev_suffixes:
    for svar in svars:
        if any((svar.sdims[k].label.startswith("p") and svar.sdims[k].label.endswith(s)) for k in svar.sdims.iterkeys()): 
            label_out=svar.label
            if svar.label.endswith(s):
                label_out=svar.label.replace(s,"")
            #print svar.label,"-->",label_out

In [88]:
# mpmoine_future_modif: liste des suffixes de noms de variables reperant un ou plusieurs niveaux pression
#multi_plev_suffixes=set(["10","19","23","27","39","3","3h","4","7c","7h","8","12"])
multi_plev_suffixes=set()
single_plev_suffixes=set(["1000","200","220","500","560","700","840","850","100"])

In [89]:
# mpmoine_future_modif: nouvelle fonction Remove_pSuffix
def Remove_pSuffix(svar,mlev_sfxs,slev_sfxs,realms):
    #
    # remove suffixes only if both suffix of svar.label *and* suffix of one of the svar.dims.label  match the search suffix
    # to avoid truncation of variable names like 'ch4' requested on 'plev19', where '4' does not stand for a plev set
    #
    import re
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    #
    #-label_out=False
    label_out=svar.label
    svar_realms=set(svar.modeling_realm.split())
    valid_realms=set(realms.split())
    if svar_realms.intersection(valid_realms):
        mvl=r.match(svar.label)
        if mvl and any(svar.label.endswith(s) for s in mlev_sfxs.union(slev_sfxs)):
            for sdim in svar.sdims.values(): 
                mdl=r.match(sdim.label)
                if mdl and mdl.group(2)==mvl.group(2): 
                     label_out=mvl.group(1)
    return label_out

In [90]:
for svar in svars:
    label_out=Remove_pSuffix(svar,multi_plev_suffixes,single_plev_suffixes,'atmos')
    if label_out!=svar.label:
        print svar.label,"--->",label_out

thetaot700 ---> thetaot
ua850 ---> ua
va200 ---> va
va850 ---> va
ua200 ---> ua
ta850 ---> ta
ta500 ---> ta
ua850 ---> ua
ta850 ---> ta
va850 ---> va
ua200 ---> ua
hus850 ---> hus
va200 ---> va
wap500 ---> wap
ta700 ---> ta
wap500 ---> wap
ta700 ---> ta
ua100 ---> ua
hus1000 ---> hus
hus100 ---> hus
ta100 ---> ta
va100 ---> va
va1000 ---> va
ua1000 ---> ua
hus1000 ---> hus
va1000 ---> va
ua1000 ---> ua
